<a href="https://colab.research.google.com/github/joostnjns/Google-Earth-Engine/blob/master/SVM_Landsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
// Query satellite image  26.996813, 14.395989
var aoi = ee.Geometry.Rectangle([14.35, 26.95, 14.51, 27.12]);
var collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1995-01-01', '1995-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);

//var rgbImage = collection.sort('CLOUD_COVER').first().select(['B3', 'B2', 'B1']);
var image_ = collection.sort('CLOUD_COVER').first();


// compute MSAVI2 using expression
var msavi2 = image_.expression(
  '((2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2)*1000',  //to be in same scale as spectral bands 
  {
    'NIR': image_.select('B4'), 
    'RED': image_.select('B3')
  }
);

//print(image_.bandNames());
image_ = image_.addBands([msavi2]);
image_ = image_.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','constant']).rename(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','msavi2']);
//print(image_.bandNames());
//Map.setCenter(14.42, 27.04, 13);


// Define the visualization parameters.
var vizParams = {
  bands: ['msavi2'],
  min: -1,
  max: 1,
  gamma: [1]
};

var vizParams2 = {
  bands: ['B3','B2','B1'],
  min: 0,
  max: 5000,
  gamma: [1,1,1]
};

// Visualize satellite image              
Map.setCenter(14.42, 27.04, 13);
//Map.addLayer(image_.clip(aoi), vizParams, 'msavi2');
Map.addLayer(image_.clip(aoi), vizParams2, 'L5_truecolor');


// Load training data
var trainData = ee.FeatureCollection('users/jneujens/trainData');
//Map.addLayer(trainData);

// Property containing land cover labels
var label = 'classname';

// Get training from polygons on image
var groundTrue = image_.sampleRegions({
  collection: trainData,
  properties: ['classvalue'],
  scale: 30
});

//print(groundTrue)


// Graph of spectral signatures training data
var options = {
  title: 'Landsat 5 surface reflectance spectral signatures',
  hAxis: {title: 'Wavelength (micrometers)'},
  vAxis: {title: 'Reflectance'},
  lineWidth: 1,
  pointSize: 4,
  series: {
    0: {color: 'grey'}, // impervious
    1: {color: 'green'}, // vegetation
    2: {color: 'red'}, // soil
}};

// Define a list of Landsat 8 wavelengths for X-axis labels.
var wavelengths = [0.47, 0.56, 0.66, 0.83, 1.56, 2.2, 11.4];

// Create the chart and set options.
var spectraChart = new ui.Chart.image.regions(
    image_.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']), trainData, ee.Reducer.mean(), 30, 'classvalue', wavelengths)
        .setChartType('ScatterChart')
        .setOptions(options);
        
//print(spectraChart)

var withRandom = groundTrue.randomColumn('random');
var split = 0.8;  // Roughly 70% training, 30% testing.
var training = withRandom.filter(ee.Filter.lt('random', split));
var testing = withRandom.filter(ee.Filter.gte('random', split));

// Create an SVM classifier with custom parameters.
var classifier = ee.Classifier.svm({
  decisionProcedure: 'Voting',
  svmType: 'C_SVC',
  kernelType: 'LINEAR',
  cost: 50
});

// Train the classifier.
var trained = classifier.train(training, 'classvalue', ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'msavi2']);

//var trainAccuracy = trained.confusionMatrix();
//print(trainAccuracy)

// Classify the image.
var classified = image_.clip(aoi).classify(trained);
var tested = testing.classify(trained);

var testAccuracy = tested.errorMatrix('classvalue', 'classification');
print(testAccuracy)
//print(testAccuracy.array())

//print(testAccuracy.accuracy())
//print(confusionMatrix.consumersAccuracy())
//print(confusionMatrix.kappa())
//print(confusionMatrix.producersAccuracy())
//print(confusionMatrix.order())

// Get a confusion matrix representing expected accuracy.
//var testAccuracy = classified.errorMatrix(training, 'classification');
//print(testAccuracy);
//print('Validation error matrix: ', testAccuracy);
//print('Validation overall accuracy: ', testAccuracy.accuracy());





// Display result
Map.addLayer(classified,
             {min: 0, max: 2, palette: ['grey', 'green','red']},
             'LC classification');
             
             
             
             
             
// a

//var vectors = classified.reduceToVectors({
//  geometry: aoi,
//  scale: 30,
//  geometryType: 'polygon',
//  eightConnected: true
//});

//Map.addLayer(vectors)